# Datensuche und Datenbereinigung
Es ist gegebenenfalls erforderlich zur Ausführen des Codes ein GoogleDrive Laufwerk zu verbinden.

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 28.5MB/s 
     |████████████████████████████████| 1.2MB 36.1MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 194kB 52.4MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
Mounted at /content/gdrive


In [2]:
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import *

In [5]:
# Google Drive Laufwerk verbinden 
# from google.colab import drive
# drive.mount('/content/drive')

# Download Images via Bing Search API


## Bereitstellen der Api



In [3]:
# API Key hinterlegen
key = os.environ.get('AZURE_SEARCH_KEY', 'e82fbd5912b84c8da3634d6f98003fda')  

In [4]:
# Test der API
results = search_images_bing(key, 'esox lucius fish')
ims = results.attrgot('contentUrl')
len(ims)

150

## Download der Bilder

In [8]:
# definieren der Fischarten und des Speicherpfades
fish_types = 'abramis_brama','ameiurus_nebulosus', 'cobitis_taenia', 'cyprinus_carpio', 'esox_lucius', 'gymnocephalus_cernua', 'salmo_trutta', 
path = Path('/content/gdrive/MyDrive/Seminar/fish_1')

Lädt die Bilder in die entsprechenden Ordner, wenn der "fish" Ordner nicht bereits exisitert. 

In [9]:
# herunterladen der Bilder, wenn nicht bereits ein "fish_1" ordner vorhanden
if not path.exists():
    path.mkdir()
    for o in fish_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} fish')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
# anzeigen der Ordnerstruktur
sorted(path.ls()) 

In [11]:
# Anzahl der Bilder 
fns = get_image_files(path)
fns
len(fns)

840

In [12]:
# Bildanzahl pro Spezies
for o in fish_types:
  dest = (path/o)
  fns = get_image_files(dest) 
  print(len(fns))

97
133
121
123
106
130
130


In [13]:
# loesche nicht als Bild zu interpretierende Dateien
failed = verify_images(fns)
failed.map(Path.unlink);

# Bereinigen der Bilder
Zum bereinigen erfolgt hier die Darstellung über ein einfaches Modell, die händische Bereinigung ist in den Ordnern erfolgt. 

In [14]:
# ohne Transformationen zur Datenbereinigung
fish = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))
dls = fish.dataloaders(path)


Die hier dargestellten Augmentationen der Daten finden später in den jeweiligen Codefiles zu den Modellen Anwendung, werden der Vollständigkeit halber hier allerdings ebenfalls aufgezeigt. 


```
# setzen der Transformationen 
batch_tfms = [*aug_transforms(size=224, min_scale=0.75, flip_vert=True, max_warp=0), 
              Normalize.from_stats(*imagenet_stats)]
bs=64
```

```
# DataLoaders mit Transformationen
fish = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460), 
    batch_tfms=batch_tfms)
dls = fish.dataloaders(path, bs=bs)
```


In [15]:
# zur Fehlersuche hilfreich
# fish.summary(path)

In [ ]:
dls.show_batch(nrows=1,ncols=3)

In [ ]:
# erstellen und trainieren des einfachen Learners zur Bereinigung der Daten
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
# starten des FastAi-Widgets zum bereinigen der Daten
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
# loeschen der Bilder mit "Delete"
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

# verschieben in anderen Ordner
#for idx in cleaner.delete(): shutil.move(str(cleaner.fns[idx]), '/content/gdrive/MyDrive/Seminar/deleted')

In [ ]:
# verschieben der umsortierten Bilder
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)